In [1]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
from datasets import DataFrameSet
from utils import *
import pandas as pd
from pathlib import Path

Available workers: 8


In [3]:
df = pd.DataFrame(get_files(DATA_DIR), columns=['filename'])

In [4]:
dfs = DataFrameSet(df)

In [5]:
dfs.df

,filename
0,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
1,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
2,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
3,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
4,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
...,...
67836,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
67837,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
67838,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...
67839,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...


In [6]:
dfs.df.filename.iloc[0]

PosixPath('/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuji/fuji/bl/2020_Herent_w28_3-30_F10_51 mm_ISO160_1-15 s_IncandescentWB_111.jpg')

In [30]:
extract_filename_info(str(dfs.df.filename.iloc[0]))

('/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuji/fuji/bl/2020_Herent_w28_3-30_F10_51 mm_ISO160_1-15 s_IncandescentWB_111.jpg',
 'bl',
 '2020_Herent_w28_3-30_F10_51 mm_ISO160_1-15 s_IncandescentWB_111',
 '2020',
 'Herent',
 'w28',
 '3-30',
 '111')

In [31]:
ser = dfs.df.filename.apply(lambda x: extract_filename_info(str(x)))
tmp = pd.DataFrame.from_records(ser, columns=['filename', 'label','name','year','location','date','xtra','idx'])
tmp

,filename,label,name,year,location,date,xtra,idx
0,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,bl,2020_Herent_w28_3-30_F10_51 mm_ISO160_1-15 s_I...,2020,Herent,w28,3-30,111
1,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,bl,2021_UNDISTORTED_Landen_w24_B_F10_51 mmmm_ISO1...,2021,UNDISTORTED_Landen,w24,B,376
2,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,bl,2021_UNDISTORTED_Landen_w24_B_F10_51 mmmm_ISO1...,2021,UNDISTORTED_Landen,w24,B,381
3,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,bl,2021_UNDISTORTED_Landen_w24_B_F10_51 mmmm_ISO1...,2021,UNDISTORTED_Landen,w24,B,386
4,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,bl,2021_UNDISTORTED_Landen_w24_B_F10_51 mmmm_ISO1...,2021,UNDISTORTED_Landen,w24,B,387
...,...,...,...,...,...,...,...,...
67836,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,wswl,2019_kampen_w31_3daysold_C_25_F10_51 mm_ISO160...,2019,kampen,w31,3daysold,67
67837,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,wswl,2019_kampen_w35_A_F10_51 mm_ISO160_1-15 s_91,2019,kampen,w35,A,91
67838,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,wswl,2019_kampen_w36_B_F10_51 mm_ISO160_1-15 s_7,2019,kampen,w36,B,7
67839,/media/kalfasyan/HGST_4TB/Datasets/pbox_vs_fuj...,wswl,2019_kampen_w39_A_F10_51 mm_ISO160_1-15 s_180,2019,kampen,w39,A,180


In [32]:
tmp.location.value_counts()

Herent                   13336
her                      12105
herent                    8141
kampen                    6706
beauv                     6237
brainelal                 4784
HER                       4259
beauvech                  3251
BEAUV                     1601
brainlal                  1521
Racour                    1050
wortel                    1004
KAMPEN                     908
BRAINELAL                  884
merchtem                   616
beavech                    370
UNDISTORTED_Mollem         221
UNDISTORTED_mollem         158
UNDISTORTED_Brainelal      146
UNDISTORTED_landen         133
racour                     109
UNDISTORTED_braine          93
UNDISTORTED_Landen          90
UNDISTORTED_brainelal       50
HerentVAL2                  30
HerentCONTROLE              23
HerentVAL1                  12
HerentVAL3                   3
Name: location, dtype: int64